**INSTALL LIBRARY**

In [1]:
!pip install --upgrade gensim

In [2]:
!pip install pyldavis==3.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.2.1-py2.py3-none-any.whl size=136164 sha256=6e8ca20db868a4226b55f6ca0c1419751c5eae0c21aae2e582fd66ff58c601d9
  Stored in directory: /root/.cache/pip/wheels/d3/35/9c/c084bb1c3a4b8a7feb32d57c88f572dcf5c3accc949478893f
Successfully built pyldavis


In [3]:
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.0 MB/s eta 0:00:00


In [4]:
!pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=060c3d7343ac7d8cfb93d3c8d4db8f6d8fb1431bdf7f99fe4a317777ca488daa
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


**MENYIAPKAN DATA**

In [5]:
import pandas as pd
import numpy as np
import nltk

In [6]:
from google.colab import files
uploades = files.upload()

Saving SI-UINSA.xlsx to SI-UINSA.xlsx


In [7]:
DATA_SI = pd.read_excel("SI-UINSA.xlsx")
DATA_SI.head()

,Dokumen,Abstrak
0,d1,"Green IT merupakan praktik perancangan, pembua..."
1,d2,Kegiatan belajar mengajar (KBM) merupakan bagi...
2,d3,E-Learning memberikan dampak signifikan dalam ...
3,d4,"Sejak bertahun lalu, hasil tes IQ menjadi pat..."
4,d5,Setiap musim hujan sejumlah wilayah di Indones...


**PREPROCESSING**

In [8]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas

DATA_SI['Abstrak'] = DATA_SI['Abstrak'].str.lower()

print('Case Folding Result : \n')
print(DATA_SI['Abstrak'].head(5))

Case Folding Result : 

0    green it merupakan praktik perancangan, pembua...
1    kegiatan belajar mengajar (kbm) merupakan bagi...
2    e-learning memberikan dampak signifikan dalam ...
3    sejak bertahun lalu, hasil tes iq  menjadi pat...
4    setiap musim hujan sejumlah wilayah di indones...
Name: Abstrak, dtype: object


In [9]:
import string
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [10]:
# ------ Tokenizing ---------

nltk.download('punkt')

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

DATA_SI['Abstrak'] = DATA_SI['Abstrak'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

DATA_SI['Abstrak'] = DATA_SI['Abstrak'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

DATA_SI['Abstrak'] = DATA_SI['Abstrak'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

DATA_SI['Abstrak'] = DATA_SI['Abstrak'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

DATA_SI['Abstrak'] = DATA_SI['Abstrak'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

DATA_SI['Abstrak'] = DATA_SI['Abstrak'].apply(remove_singl_char)

# NLTK word tokenize
def word_tokenize_wrapper(text):
    return word_tokenize(text)

DATA_SI['Abstrak_tokens'] = DATA_SI['Abstrak'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n')
print(DATA_SI['Abstrak_tokens'].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Tokenizing Result : 

0    [green, it, merupakan, praktik, perancangan, p...
1    [kegiatan, belajar, mengajar, kbm, merupakan, ...
2    [elearning, memberikan, dampak, signifikan, da...
3    [sejak, bertahun, lalu, hasil, tes, iq, menjad...
4    [setiap, musim, hujan, sejumlah, wilayah, di, ...
Name: Abstrak_tokens, dtype: object


In [11]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

DATA_SI['Abstrak_tokens_fdist'] = DATA_SI['Abstrak_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n')
print(DATA_SI['Abstrak_tokens_fdist'].head().apply(lambda x : x.most_common()))


Frequency Tokens : 

0    [(it, 9), (yang, 7), (dan, 6), (green, 5), (pe...
1    [(pembelajaran, 8), (yang, 8), (dari, 7), (dal...
2    [(dan, 12), (hipotesis, 9), (signifikan, 6), (...
3    [(kemampuan, 4), (iq, 3), (menjadi, 2), (quoti...
4    [(banjir, 7), (untuk, 5), (dengan, 5), (dan, 5...
Name: Abstrak_tokens_fdist, dtype: object


In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang',
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih',
                       'si', 'tau', 'pt', 'hasil' 'tdk', 'tuh', 'utk', 'ya',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'saya', 'yang', 'dan', 'aja', 'biar', 'bikin', 'nan', 'yah', 'tuh', 'nih', 'sih', 'dan', 'jika', 'itu', 'untuk', 'ke', 'pada', 'terus', 'saya'])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

DATA_SI['Abstrak_tokens_WSW'] = DATA_SI['Abstrak_tokens'].apply(stopwords_removal)

print(DATA_SI['Abstrak_tokens_WSW'].head())

0    [green, it, praktik, perancangan, pembuatan, p...
1    [kegiatan, belajar, mengajar, kbm, proses, pem...
2    [elearning, dampak, signifikan, sistem, pendid...
3    [bertahun, hasil, tes, iq, patokan, mengukur, ...
4    [musim, hujan, wilayah, indonesia, dihimbau, w...
Name: Abstrak_tokens_WSW, dtype: object


**NORMALISASI**

In [16]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in DATA_SI['Abstrak_tokens_WSW']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print(len(term_dict))


2344


In [17]:
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)

    # untuk melihat hasilnya silahkan jalankan baris di bawah ini
    print(term,":" ,term_dict[term])

green : green
it : it
praktik : praktik
perancangan : ancang
pembuatan : buat
penggunaan : guna
perangkat : perangkat
komputer : komputer
printer : printer
monitor : monitor
server : server
penyimpanan : simpan
sistem : sistem
jaringan : jaring
efisien : efisien
efektif : efektif
menimbulkan : timbul
dampak : dampak
serius : serius
lingkungan : lingkung
penelitian : teliti
terfokus : fokus
use : use
of : of
system : system
ramah : ramah
mengurangi : kurang
bertujuan : tuju
menentukan : tentu
indikatorindikator : indikatorindikator
signifikan : signifikan
kesiapan : kesiap
implementasi : implementasi
prima : prima
visi : visi
globalindo : globalindo
berdasarkan : dasar
framework : framework
greadiness : greadiness
model : model
usulan : usul
indonesia : indonesia
hasil : hasil
exploratory : exploratory
factor : factor
analysis : analysis
data : data
diambil : ambil
teknik : teknik
total : total
sampling : sampling
analisis : analisis
kuantitatif : kuantitatif
deskriptif : deskriptif
ind

In [18]:
# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

DATA_SI['Abstrak_tokens_stemmed'] = DATA_SI['Abstrak_tokens_WSW'].swifter.apply(get_stemmed_term)

print(DATA_SI['Abstrak_tokens_stemmed'])

Pandas Apply:   0%|          | 0/79 [00:00<?, ?it/s]

0     [green, it, praktik, ancang, buat, guna, peran...
1     [giat, ajar, ajar, kbm, proses, ajar, langsung...
2     [elearning, dampak, signifikan, sistem, didik,...
3     [tahun, hasil, tes, iq, patok, ukur, cerdas, s...
4     [musim, hujan, wilayah, indonesia, dihimbau, w...
                            ...                        
74    [fokus, computational, thinking, letak, aspek,...
75    [perintah, indonesia, terap, sistem, perintah,...
76    [kecap, kenari, salah, usaha, pasok, distribus...
77    [qris, metode, bayar, alternatif, kurang, kont...
78    [minta, beras, tingkat, jadi, beras, salah, ko...
Name: Abstrak_tokens_stemmed, Length: 79, dtype: object


In [38]:
#stopwords #2

from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["ada", "hasil", "jadi", "pt", "buat", "ajar", "hari", "tahun", "artikel",
                       "metode", "guna", "the"])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

DATA_SI['Abstrak_tokens_stemmed2'] = DATA_SI['Abstrak_tokens_stemmed'].apply(stopwords_removal)

print(DATA_SI['Abstrak_tokens_stemmed2'].head())

0    [green, it, praktik, ancang, perangkat, komput...
1    [giat, kbm, proses, langsung, pengaruh, didik,...
2    [elearning, dampak, signifikan, sistem, didik,...
3    [tes, iq, patok, ukur, cerdas, sorang, iq, int...
4    [musim, hujan, wilayah, indonesia, dihimbau, w...
Name: Abstrak_tokens_stemmed2, dtype: object


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
for i in range(len(DATA_SI)):
        a=DATA_SI.iloc[i][6]
        document.append(a)

document[0:5]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['permintaan', 'beras', 'meningkat', 'menjadikan', 'beras']

In [40]:
doc_clean = DATA_SI['Abstrak_tokens_stemmed2']
doc_clean

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0     [green, it, praktik, ancang, perangkat, komput...
1     [giat, kbm, proses, langsung, pengaruh, didik,...
2     [elearning, dampak, signifikan, sistem, didik,...
3     [tes, iq, patok, ukur, cerdas, sorang, iq, int...
4     [musim, hujan, wilayah, indonesia, dihimbau, w...
                            ...                        
74    [fokus, computational, thinking, letak, aspek,...
75    [perintah, indonesia, terap, sistem, perintah,...
76    [kecap, kenari, salah, usaha, pasok, distribus...
77    [qris, bayar, alternatif, kontak, fisik, manfa...
78    [beras, tingkat, beras, salah, komoditas, pang...
Name: Abstrak_tokens_stemmed2, Length: 79, dtype: object

**LDA GENSIM**

In [41]:
import gensim
from gensim import corpora

dictionary = corpora.Dictionary(doc_clean)
print(dictionary)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

Dictionary<1879 unique tokens: ['ambil', 'analisis', 'analysis', 'ancang', 'attitude']...>


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

total_topics = 3 # jumlah topik yang akan di extract
number_words = 10 # jumlah kata per topik

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [43]:
# Running and Trainign LDA model on the document term matrix.
lda_model = Lda(doc_term_matrix, num_topics=total_topics, id2word = dictionary, passes=50)

lda_model.show_topics(num_topics=total_topics, num_words=number_words)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.021*"teliti" + 0.019*"pengaruh" + 0.013*"data" + 0.012*"model" + 0.011*"aplikasi" + 0.011*"signifikan" + 0.011*"variabel" + 0.008*"positif" + 0.008*"nilai" + 0.008*"tingkat"'),
 (1,
  '0.018*"teliti" + 0.010*"surabaya" + 0.010*"digital" + 0.009*"informasi" + 0.008*"of" + 0.007*"pengaruh" + 0.007*"layan" + 0.006*"anak" + 0.006*"nilai" + 0.006*"variabel"'),
 (2,
  '0.028*"teliti" + 0.012*"teknologi" + 0.011*"risiko" + 0.011*"informasi" + 0.011*"proses" + 0.010*"data" + 0.010*"kembang" + 0.010*"model" + 0.009*"nilai" + 0.009*"sistem"')]

In [44]:
# Word Count of Topic Keywords

from collections import Counter
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in doc_clean for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df_imp_wcount = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])
print(df_imp_wcount)

          word  topic_id  importance  word_count
0       teliti         0    0.021115         225
1     pengaruh         0    0.019117         104
2         data         0    0.012808          98
3        model         0    0.011890          88
4     aplikasi         0    0.011308          62
5   signifikan         0    0.010894          59
6     variabel         0    0.010690          59
7      positif         0    0.008458          43
8        nilai         0    0.008164          79
9      tingkat         0    0.007581          60
10      teliti         1    0.017743         225
11    surabaya         1    0.009644          33
12     digital         1    0.009635          38
13   informasi         1    0.009263          80
14          of         1    0.007523          32
15    pengaruh         1    0.007206         104
16       layan         1    0.006898          52
17        anak         1    0.006176          14
18       nilai         1    0.006080          79
19    variabel      

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [45]:
from google.colab import drive
drive.mount('drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [46]:
#simpan ke google drive
df_imp_wcount.to_csv('df_imp_wcount.csv')
!cp df_imp_wcount.csv "drive/My Drive/LDA/"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [47]:
#Dominant topic and its percentage contribution in each topic
def format_topics_sentences(ldamodel=None, corpus=doc_term_matrix, texts=document):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [48]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=doc_clean)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
print(df_dominant_topic.head(10))

   Document_No  Dominant_Topic  Topic_Perc_Contrib  \
0            0               0              0.9930   
1            1               2              0.5377   
2            2               0              0.9932   
3            3               1              0.9852   
4            4               2              0.9931   
5            5               1              0.9926   
6            6               0              0.9956   
7            7               2              0.9941   
8            8               2              0.9909   
9            9               2              0.9920   

                                            Keywords  \
0  teliti, pengaruh, data, model, aplikasi, signi...   
1  teliti, teknologi, risiko, informasi, proses, ...   
2  teliti, pengaruh, data, model, aplikasi, signi...   
3  teliti, surabaya, digital, informasi, of, peng...   
4  teliti, teknologi, risiko, informasi, proses, ...   
5  teliti, surabaya, digital, informasi, of, peng...   
6  teliti, pe

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [49]:
#simpan ke google drive
df_dominant_topic.to_csv('df_dominant_topic.csv')
!cp df_dominant_topic.csv "drive/My Drive/LDA/"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**VISUALISASI**

In [50]:
import pyLDAvis.gensim
import pickle
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [51]:
import os
LDAvis_data_filepath = os.path.join('ldavis_prepared_'+str(total_topics))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [52]:
corpus = [dictionary.doc2bow(text) for text in doc_clean]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [53]:
# proses
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [54]:
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [55]:
pyLDAvis.save_html(LDAvis_prepared, 'drive/My Drive/LDA/ldavis_prepared_'+ str(total_topics) +'.html')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [56]:
# proses
LDAvis_prepared

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.086867  0.090753       1        1  45.033578
2     -0.128317  0.024279       2        1  34.181196
1      0.041449 -0.115032       3        1  20.785226, topic_info=           Term       Freq      Total Category  logprob  loglift
459      risiko  36.000000  36.000000  Default  30.0000  30.0000
98     pengaruh  87.000000  87.000000  Default  29.0000  29.0000
197    variabel  49.000000  49.000000  Default  28.0000  28.0000
303    surabaya  26.000000  26.000000  Default  27.0000  27.0000
168     digital  30.000000  30.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
52   signifikan   7.643901  49.733735   Topic3  -5.4002  -0.3018
503    perintah   7.039932  25.667451   Topic3  -5.4825   0.2773
118         uji   7.435710  45.891885   Topic3  -5.4278  -0.2491
12         data   7.569061  81.972124   Topic3  -5.4100  -0.8114
354     website   6.824085  20.779976   Topic3  -5.5137   0.4574

[212 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
956       1  0.887558  akademik
439       2  0.934252      amal
487       1  0.076628      aman
487       2  0.306512      aman
487       3  0.613023      aman
...     ...       ...       ...
354       1  0.384986   website
354       2  0.288740   website
354       3  0.336863   website
1171      3  0.845802       wfh
634       3  0.845802    window

[268 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])